# Explore Kernel Density Estimate Fitting on WNS 'Endemic Area'

In [1]:
import os
import ipywidgets as widgets
from ipywidgets import interact

from osgeo import gdal
import geopandas as gpd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
from shapely.geometry import mapping, Polygon, MultiPolygon, LineString, Point

In [2]:
# wns_fname = os.path.join('data', "WNS_Status_Provisional_8_30_2019.shp")
wns_fname = os.path.join('data', "WNS_Status_Provisional_determination.shp")

wns = gpd.GeoDataFrame.from_file(wns_fname)
wns['year'] = wns.map_year


In [3]:
target_crs = {'proj': 'aea',
 'lat_1': 29.5,
 'lat_2': 45.5,
 'lat_0': 23,
 'lon_0': -96,
 'x_0': 0,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'm',
 'no_defs': True}

In [4]:
states_fname = os.path.join('data', "cb_2017_us_state_20m.shp")
states = gpd.read_file(states_fname)
states_aea = states.to_crs(target_crs)
conus_states = states_aea[~states_aea.NAME.isin(['Alaska', 'Hawaii', 'Puerto Rico'])]

bounds = conus_states.bounds
x_bounds = [bounds.minx.min(), bounds.maxx.max()]
y_bounds = [bounds.miny.min(), bounds.maxy.max()]

In [5]:
wns_centroid = wns.copy().to_crs({'init': 'epsg:4326'})
wns_centroid.geometry = wns_centroid.geometry.centroid

centroids = wns.centroid
centroids = centroids.to_crs(target_crs)
wns['x'] = centroids.x
wns['y'] = centroids.y

wns['area'] = centroids.to_crs(target_crs).geometry.area
wns['area_weight'] = wns.area/wns.area.max()
# wns.head(5)

In [6]:
def reclass_as_pcnt(Z):
    uniques = np.unique(Z)[::-1]
    z_reclass = np.copy(Z)
    total_dens = Z.sum()
    cum_area = 0.0
    for u in uniques:
        cum_area += np.count_nonzero(Z==u)*u
        z_reclass[Z==u] = cum_area/total_dens
    return z_reclass
    #area_lookup[unique] = np.count_nonzero(z==unique)

def create_kernel(x, y, X, Y, factor=1.2, weights=None):
    
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([x, y])
    kernel = gaussian_kde(values, weights=weights)#, bw_method='silverman')#, bw_method=0.15/np.asarray(values).std(ddof=1))
#     kernel.silverman_factor()
#     kernel.set_bandwidth(bw_method='silverman')
    kernel.set_bandwidth( factor)
    # kernel.set_bandwidth(bw_method=bw_method)
    # kernel.set_bandwidth(bw_method=0.15/np.asarray(values).std(ddof=1))
    Z = np.reshape(kernel(positions).T, X.shape)
    return reclass_as_pcnt(Z)

def create_kernel_contours(x, y, z, levels=[0.5, 0.75, 0.95]):
    cset = plt.contour(x, y, z, levels=levels, colors=['red', 'white', 'blue'])
    return cset



def plot_one(x, y, X, Y, Z, title='', isopleth=0.75):
    fig = plt.figure(figsize=(25, 15))
    ax = fig.add_subplot(111)

    cset = create_kernel_contours(X, Y, Z, levels=[isopleth])   
    
#     ax.imshow(np.rot90(Z), cmap=plt.cm.Reds_r, 
#                extent=[X.min(), X.max(), Y.min(), Y.max()], alpha=0.6)

    hull = Polygon(zip(x, y)).convex_hull
    pnts = np.array(np.asarray(hull.exterior)).transpose()
    
#     ax.plot(pnts[0],pnts[1])
    
#     ax.plot(x, y, 'k.', markersize=6, alpha=0.4)
#     ax.set_xlim([X.min(), X.max()])
#     ax.set_ylim([Y.min(), Y.max()])
    ax.set_xlim([-3000000, 3000000])
    ax.set_ylim([0, 3700000])
#     plt.colorbar()
    states_aea.plot(color='None',  edgecolor='black', ax=ax, alpha=0.4)
    plt.title(title,fontsize=30)
#     plt.show()
    ax.set_aspect('equal')
    return ax

In [7]:
# copied from https://gist.github.com/tillahoffmann/f844bce2ec264c1c8cb5

import numpy as np
from scipy.spatial.distance import cdist

class gaussian_kde(object):
    """Representation of a kernel-density estimate using Gaussian kernels.

    Kernel density estimation is a way to estimate the probability density
    function (PDF) of a random variable in a non-parametric way.
    `gaussian_kde` works for both uni-variate and multi-variate data.   It
    includes automatic bandwidth determination.  The estimation works best for
    a unimodal distribution; bimodal or multi-modal distributions tend to be
    oversmoothed.

    Parameters
    ----------
    dataset : array_like
        Datapoints to estimate from. In case of univariate data this is a 1-D
        array, otherwise a 2-D array with shape (# of dims, # of data).
    bw_method : str, scalar or callable, optional
        The method used to calculate the estimator bandwidth.  This can be
        'scott', 'silverman', a scalar constant or a callable.  If a scalar,
        this will be used directly as `kde.factor`.  If a callable, it should
        take a `gaussian_kde` instance as only parameter and return a scalar.
        If None (default), 'scott' is used.  See Notes for more details.
    weights : array_like, shape (n, ), optional, default: None
        An array of weights, of the same shape as `x`.  Each value in `x`
        only contributes its associated weight towards the bin count
        (instead of 1).

    Attributes
    ----------
    dataset : ndarray
        The dataset with which `gaussian_kde` was initialized.
    d : int
        Number of dimensions.
    n : int
        Number of datapoints.
    neff : float
        Effective sample size using Kish's approximation.
    factor : float
        The bandwidth factor, obtained from `kde.covariance_factor`, with which
        the covariance matrix is multiplied.
    covariance : ndarray
        The covariance matrix of `dataset`, scaled by the calculated bandwidth
        (`kde.factor`).
    inv_cov : ndarray
        The inverse of `covariance`.

    Methods
    -------
    kde.evaluate(points) : ndarray
        Evaluate the estimated pdf on a provided set of points.
    kde(points) : ndarray
        Same as kde.evaluate(points)
    kde.pdf(points) : ndarray
        Alias for ``kde.evaluate(points)``.
    kde.set_bandwidth(bw_method='scott') : None
        Computes the bandwidth, i.e. the coefficient that multiplies the data
        covariance matrix to obtain the kernel covariance matrix.
        .. versionadded:: 0.11.0
    kde.covariance_factor : float
        Computes the coefficient (`kde.factor`) that multiplies the data
        covariance matrix to obtain the kernel covariance matrix.
        The default is `scotts_factor`.  A subclass can overwrite this method
        to provide a different method, or set it through a call to
        `kde.set_bandwidth`.

    Notes
    -----
    Bandwidth selection strongly influences the estimate obtained from the KDE
    (much more so than the actual shape of the kernel).  Bandwidth selection
    can be done by a "rule of thumb", by cross-validation, by "plug-in
    methods" or by other means; see [3]_, [4]_ for reviews.  `gaussian_kde`
    uses a rule of thumb, the default is Scott's Rule.

    Scott's Rule [1]_, implemented as `scotts_factor`, is::

        n**(-1./(d+4)),

    with ``n`` the number of data points and ``d`` the number of dimensions.
    Silverman's Rule [2]_, implemented as `silverman_factor`, is::

        (n * (d + 2) / 4.)**(-1. / (d + 4)).

    Good general descriptions of kernel density estimation can be found in [1]_
    and [2]_, the mathematics for this multi-dimensional implementation can be
    found in [1]_.

    References
    ----------
    .. [1] D.W. Scott, "Multivariate Density Estimation: Theory, Practice, and
           Visualization", John Wiley & Sons, New York, Chicester, 1992.
    .. [2] B.W. Silverman, "Density Estimation for Statistics and Data
           Analysis", Vol. 26, Monographs on Statistics and Applied Probability,
           Chapman and Hall, London, 1986.
    .. [3] B.A. Turlach, "Bandwidth Selection in Kernel Density Estimation: A
           Review", CORE and Institut de Statistique, Vol. 19, pp. 1-33, 1993.
    .. [4] D.M. Bashtannyk and R.J. Hyndman, "Bandwidth selection for kernel
           conditional density estimation", Computational Statistics & Data
           Analysis, Vol. 36, pp. 279-298, 2001.

    Examples
    --------
    Generate some random two-dimensional data:

    >>> from scipy import stats
    >>> def measure(n):
    >>>     "Measurement model, return two coupled measurements."
    >>>     m1 = np.random.normal(size=n)
    >>>     m2 = np.random.normal(scale=0.5, size=n)
    >>>     return m1+m2, m1-m2

    >>> m1, m2 = measure(2000)
    >>> xmin = m1.min()
    >>> xmax = m1.max()
    >>> ymin = m2.min()
    >>> ymax = m2.max()

    Perform a kernel density estimate on the data:

    >>> X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    >>> positions = np.vstack([X.ravel(), Y.ravel()])
    >>> values = np.vstack([m1, m2])
    >>> kernel = stats.gaussian_kde(values)
    >>> Z = np.reshape(kernel(positions).T, X.shape)

    Plot the results:

    >>> import matplotlib.pyplot as plt
    >>> fig = plt.figure()
    >>> ax = fig.add_subplot(111)
    >>> ax.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,
    ...           extent=[xmin, xmax, ymin, ymax])
    >>> ax.plot(m1, m2, 'k.', markersize=2)
    >>> ax.set_xlim([xmin, xmax])
    >>> ax.set_ylim([ymin, ymax])
    >>> plt.show()

    """
    def __init__(self, dataset, bw_method=None, weights=None):
        self.dataset = np.atleast_2d(dataset)
        if not self.dataset.size > 1:
            raise ValueError("`dataset` input should have multiple elements.")
        self.d, self.n = self.dataset.shape
            
        if weights is not None:
            self.weights = weights / np.sum(weights)
        else:
            self.weights = np.ones(self.n) / self.n
            
        # Compute the effective sample size 
        # http://surveyanalysis.org/wiki/Design_Effects_and_Effective_Sample_Size#Kish.27s_approximate_formula_for_computing_effective_sample_size
        self.neff = 1.0 / np.sum(self.weights ** 2)

        self.set_bandwidth(bw_method=bw_method)

    def evaluate(self, points):
        """Evaluate the estimated pdf on a set of points.

        Parameters
        ----------
        points : (# of dimensions, # of points)-array
            Alternatively, a (# of dimensions,) vector can be passed in and
            treated as a single point.

        Returns
        -------
        values : (# of points,)-array
            The values at each point.

        Raises
        ------
        ValueError : if the dimensionality of the input points is different than
                     the dimensionality of the KDE.

        """
        points = np.atleast_2d(points)

        d, m = points.shape
        if d != self.d:
            if d == 1 and m == self.d:
                # points was passed in as a row vector
                points = np.reshape(points, (self.d, 1))
                m = 1
            else:
                msg = "points have dimension %s, dataset has dimension %s" % (d,
                    self.d)
                raise ValueError(msg)

        # compute the normalised residuals
        chi2 = cdist(points.T, self.dataset.T, 'mahalanobis', VI=self.inv_cov) ** 2
        # compute the pdf
        result = np.sum(np.exp(-.5 * chi2) * self.weights, axis=1) / self._norm_factor

        return result

    __call__ = evaluate

    def scotts_factor(self):
        return np.power(self.neff, -1./(self.d+4))

    def silverman_factor(self):
        return np.power(self.neff*(self.d+2.0)/4.0, -1./(self.d+4))

    #  Default method to calculate bandwidth, can be overwritten by subclass
    covariance_factor = scotts_factor

    def set_bandwidth(self, bw_method=None):
        """Compute the estimator bandwidth with given method.

        The new bandwidth calculated after a call to `set_bandwidth` is used
        for subsequent evaluations of the estimated density.

        Parameters
        ----------
        bw_method : str, scalar or callable, optional
            The method used to calculate the estimator bandwidth.  This can be
            'scott', 'silverman', a scalar constant or a callable.  If a
            scalar, this will be used directly as `kde.factor`.  If a callable,
            it should take a `gaussian_kde` instance as only parameter and
            return a scalar.  If None (default), nothing happens; the current
            `kde.covariance_factor` method is kept.

        Notes
        -----
        .. versionadded:: 0.11

        Examples
        --------
        >>> x1 = np.array([-7, -5, 1, 4, 5.])
        >>> kde = stats.gaussian_kde(x1)
        >>> xs = np.linspace(-10, 10, num=50)
        >>> y1 = kde(xs)
        >>> kde.set_bandwidth(bw_method='silverman')
        >>> y2 = kde(xs)
        >>> kde.set_bandwidth(bw_method=kde.factor / 3.)
        >>> y3 = kde(xs)

        >>> fig = plt.figure()
        >>> ax = fig.add_subplot(111)
        >>> ax.plot(x1, np.ones(x1.shape) / (4. * x1.size), 'bo',
        ...         label='Data points (rescaled)')
        >>> ax.plot(xs, y1, label='Scott (default)')
        >>> ax.plot(xs, y2, label='Silverman')
        >>> ax.plot(xs, y3, label='Const (1/3 * Silverman)')
        >>> ax.legend()
        >>> plt.show()

        """
        if bw_method is None:
            pass
        elif bw_method == 'scott':
            self.covariance_factor = self.scotts_factor
        elif bw_method == 'silverman':
            self.covariance_factor = self.silverman_factor
        elif np.isscalar(bw_method):
            self._bw_method = 'use constant'
            self.covariance_factor = lambda: self.factor / bw_method
        elif callable(bw_method):
            self._bw_method = bw_method
            self.covariance_factor = lambda: self._bw_method(self)
        else:
            msg = "`bw_method` should be 'scott', 'silverman', a scalar " \
                  "or a callable."
            raise ValueError(msg)

        self._compute_covariance()

    def _compute_covariance(self):
        """Computes the covariance matrix for each Gaussian kernel using
        covariance_factor().
        """
        self.factor = self.covariance_factor()
        # Cache covariance and inverse covariance of the data
        if not hasattr(self, '_data_inv_cov'):
            # Compute the mean and residuals
            _mean = np.sum(self.weights * self.dataset, axis=1)
            _residual = (self.dataset - _mean[:, None])
            # Compute the biased covariance
            self._data_covariance = np.atleast_2d(np.dot(_residual * self.weights, _residual.T))
            # Correct for bias (http://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_covariance)
            self._data_covariance /= (1 - np.sum(self.weights ** 2))
            self._data_inv_cov = np.linalg.inv(self._data_covariance)

        self.covariance = self._data_covariance * self.factor**2
        self.inv_cov = self._data_inv_cov / self.factor**2
        self._norm_factor = np.sqrt(np.linalg.det(2*np.pi*self.covariance)) #* self.n

In [8]:
cache = {}

def interactive_plot(factor, iso=0.75, year=2017, 
                     add_counties=False, 
                     add_centroids=True,
                     add_surface=True,
                     weight_by_county_area=True,
                     weight_by_year=True):
    if year in cache:
        X, Y, data = cache[year]['data']
    else:
        cache[year] = {}
        cache[year]['kernels'] = {}
        data = wns[wns.year <= year]
        data.years_pres = data.year.max()-data.year
        data = data[~data.determinat.isin(['Pd negative', 'WNS negative'])]
        
        pad=500000
        X, Y = np.mgrid[x_bounds[0]-pad:x_bounds[1]+pad:100j, y_bounds[0]-pad:y_bounds[1]+pad:100j]
        cache[year]['data'] = (X, Y, data)
    
    
    if weight_by_year:
        year_weights = gen_year_weights()
        weights = data.years_pres.apply(lambda x: year_weights[x])
    else:
        weights = np.ones(data.area_weight.shape)
        
    weights /= np.sum(weights)
    
    if weight_by_county_area:
        county_weights = data.area_weight
        county_weights /= np.sum(county_weights)
        weights *= county_weights
        
#     weights /= np.sum(weights)
    weights = list(weights)

    Z = create_kernel(data.x, data.y, X, Y, factor, weights=weights)
    cache[year]['kernels'][factor] = Z
        
        
    ax = plot_one(data.x, data.y, X, Y, Z, isopleth=iso, title=f"WNS {year}-{year+1}")
    
    if add_surface:
        ax.imshow(np.rot90(Z), cmap=plt.cm.Reds_r, 
           extent=[X.min(), X.max(), Y.min(), Y.max()], alpha=0.6)
    
    if add_counties:
        data.to_crs(states_aea.crs).plot(column='year', cmap='plasma', ax=ax, legend=True)

    if add_centroids:
        data.to_crs(states_aea.crs).centroid.plot(color='grey', ax=ax)

        

In [9]:
bw_w = widgets.FloatSlider(
    value=1.2,
    min=0.1,
    max=4.0,
    step=0.1,
    description="BW Factor")

bw_w.continuous_update = False

In [10]:
year_w = widgets.IntSlider(
    value=2014,
    min=wns.year.min()+1,
    max=wns.year.max(),
    step=1,
    description="Year")

In [11]:
iso_w = widgets.FloatSlider(
    value=0.75,
    min=0.05,
    max=.99,
    step=0.05,
    description="Isopleth")

iso_w.continuous_update = False

In [12]:
show_counties = widgets.Checkbox(value=False, description='Show Counties')
show_centroids = widgets.Checkbox(value=False, description='Show Centroids')
show_kde = widgets.Checkbox(value=False, description='Show KDE Surface')
weight_by_county = widgets.Checkbox(value=True, description='Weight by county areas')
weight_by_year = widgets.Checkbox(value=True, description='Weight by years since detection')



### In order to weight counties we need to decide on a function to use for weighting.
#### Let's start with a simple number of years raised to a power function, as it allows us to explore a range of curves

*move the slider below to set the shape of this curve*
* 0.0 is no weight
* 1.0 is linear

***Note: Changes to this curve do not update the map until one of the widgets below is triggered***

In [13]:
weight_f = widgets.FloatSlider(
    value=0,
    min=0.0,
    max=10.0,
    step=0.1,
    description="Weight Exponent",
    layout={'width': '500px'},
    style={'description_width': 'initial'},
    continuous_update=False)

skip_last_years = widgets.IntSlider(
    value=4,
    min=0.0,
    max=10.0,
    description="Number of years to skip",
    layout={'width': '500px'},
    style={'description_width': 'initial'})

def exponential(x, factor=0.0):
    return x**factor

wns['years_pres'] = wns.year.max()-wns.year

def gen_year_weights():
    years = list(range(0, wns.years_pres.max() + 1))
    
    max_val = (years[-1]-skip_last_years.value)**weight_f.value
    
    weight_lookup = {}
    for year in years:
        if year < skip_last_years.value:
            weight_lookup[year] = 0
        elif year >= skip_last_years.value:
            weight_lookup[year] = ((year-skip_last_years.value)**weight_f.value)/max_val
       
    return weight_lookup

def show_weight_curve(factor=0, skip_years=4):
   
    weights_lookup = gen_year_weights()
    
    f, ax1 = plt.subplots(1, 1, figsize=(6,4))
    
    ax1.scatter(weights_lookup.keys(), weights_lookup.values(), color='r', linewidth=10)
    ax1.set_title('County Weights by Years since detection')
    ax1.set_xlabel('Years since Detection (n)')
    ax1.set_ylabel('Weight')
    
    ax1.set_ylim(-0.05, 1.1)
    
#     tex = r'($n-' +str(skip_years) + r')^{' + str(factor) + '}$'
#     ax1.text(1, 0.8, tex, fontsize=20, va='bottom')
    
    plt.tight_layout()

_ = interact(show_weight_curve, factor=weight_f, skip_years=skip_last_years)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Weight Exponent', layout=La…

## Once the weight function is finished used the sliders below to see the effect on our KDE

In [14]:
%matplotlib inline

_ = interact(interactive_plot, factor=bw_w, iso=iso_w, year=year_w, 
             add_counties=show_counties,
             add_centroids=show_centroids,
             add_surface=show_kde,
             weight_by_county_area=weight_by_county,
             weight_by_years=weight_by_year)

interactive(children=(FloatSlider(value=1.2, continuous_update=False, description='BW Factor', max=4.0, min=0.…

In [15]:
from IPython.core.display import display, clear_output

In [16]:
import numpy as np
from shapely.geometry import polygon
import pandas as pd
import geopandas
from shapely import geometry

In [17]:
# Save Shapefiles
def save_kdes():
    rows = []
    for year in sorted(wns.year.unique())[1:]:
#         print(year)

        data = wns[wns.year <= year]
        data = data[~data.determinat.isin(['Pd negative', 'WNS negative'])]
    
        pad=500000
        X, Y = np.mgrid[x_bounds[0]-pad:x_bounds[1]+pad:100j, y_bounds[0]-pad:y_bounds[1]+pad:100j]

        if weight_by_county.value:
            year_weights = gen_year_weights()
            weights = data.years_pres.apply(lambda x: year_weights[x])
        else:
            weights = np.ones(data.area_weight.shape)

        weights /= np.sum(weights)

        if weight_by_county.value:
            county_weights = data.area_weight
            county_weights /= np.sum(county_weights)
            weights *= county_weights

        #     weights /= np.sum(weights)
        weights = list(weights)

        Z = create_kernel(data.x, data.y, X, Y, factor=bw_w.value, weights=weights)
        cset = create_kernel_contours(X, Y, Z, levels=[iso_w.value])

        p = cset.collections[0].get_paths()[0]
        v = p.vertices
        x = v[:,0]
        y = v[:,1]


        polys = []
        for col in cset.collections:
    #         print('col')
            # Loop through all polygons that have the same intensity level
            for contour_path in col.get_paths():
    #             print('c_path')
                # Create the polygon for this intensity level
                # The first polygon in the path is the main one, the following ones are "holes"
                for ncp, cp in enumerate(contour_path.to_polygons()):
                    x = np.array(cp)[:,0]
                    y = np.array(cp)[:,1]
                    new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])

                    hole = False
                    for p in polys:
                        if p.contains(new_shape):
                            p = polys.pop(polys.index(p))
                            hole = True
                            new_shape = p.difference(new_shape)

                    polys.append(new_shape)

        mp = MultiPolygon(polys)             


        row ={'year':year,
              'iso':iso_w.value,
              'bw_factor':bw_w.value,
              'skip_years':skip_last_years.value,
              'weight_exp':weight_f.value,
              'geometry':mp}
        rows.append(row)

    df = pd.DataFrame(rows)

    gdf = geopandas.GeoDataFrame(
        df, geometry=df.geometry)
    gdf.crs = target_crs

    gdf.to_file(r"data\wns_endemic.shp")
    plt.close()
    return gdf

## Make a prettier leaflet map of these data

In [18]:
import folium
from folium import plugins


import ipywidgets as widgets
from ipywidgets import interact
import matplotlib as mpl
from matplotlib import pyplot as plt

import seaborn as sns

In [19]:
def get_county_style(feature):
    
    y = feature['properties']['map_year']
    determination = feature['properties']['determinat']
#     print(y, suspect)
    
    if this_year-y >= skip_last_years.value:
        fillcolor = year_colors[-1]
        linewidth = .5
        opacity = 0.2
        fill_opacity = 0.4
    else:
        fillcolor = year_colors[this_year-y]
        linewidth = .5
        opacity = 0.9
        fill_opacity = 0.7
        
    wns_outline_color = 'grey'
    pd_outline_color = 'grey'       
        
    if determination == 'WNS positive':
        return {'fillColor':fillcolor,
               'weight':linewidth,
               'color':wns_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}
    if determination == 'WNS suspect':
        return {'fillColor':fillcolor,
#                 'fillPattern':suspect, 
               'weight':linewidth,
               'color':wns_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}
    if determination == 'WNS negative':
        return {'fillColor':fillcolor,
#                 'fillPattern':suspect, 
               'weight':linewidth+2,
               'color':wns_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}
    if determination == 'Pd positive':
        return {'fillColor':fillcolor,
               'weight':linewidth,
               'color':pd_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}
    if determination == 'Pd presumed':
        return {'fillColor':fillcolor,
#                 'fillPattern':suspect, 
               'weight':linewidth,
               'color':pd_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}
    if determination == 'Pd negative':
        return {'fillColor':fillcolor,
               'weight':linewidth+2,
               'color':wns_outline_color,
               'opacity':opacity,
               'fillOpacity':fill_opacity}

In [20]:
def make_map():
    m = folium.Map([39, -85], zoom_start=5, tiles='cartodbpositron', control_scale=True)
    # for p in year_suspect:
    #     p.add_to(m)
    # endemic_suspect.add_to(m)
    global this_year
    this_year = year_dropdown.value

    wns_subset = wns[wns.map_year <= this_year]

    highlight_style = lambda feature: {'color':'black',
                                          'weight':3, 'opacity':0.7}

    gj = folium.GeoJson(kdes[kdes.year==this_year-skip_last_years.value], 
                        name=f"Endemic Area: <= {this_year-skip_last_years.value}",
                       style_function=lambda feature: {'fillColor':endemic_color.value, 
                                                       'weight':0.5,
                                                       'color':'grey',
                                                       'opacity':0.5,
                                                       'fillOpacity':0.75}).add_to(m)

    #Add endemic counties to Map
    endemic_counties = wns_subset#[((wns2.year <= this_year-n_years))]
    endemic_tool_tip_geojson = folium.features.GeoJsonTooltip(
            fields=['map_year', 'determinat', 'countyname', 'stateprov'],
            aliases=["year", "White Nose STATUS", "County", "State/Province"],
            labels=True,
            localize=False,
            style=None,
        )
    folium.GeoJson(endemic_counties, name=f"Endemic Area Counties",
                           tooltip=endemic_tool_tip_geojson,
                           style_function=get_county_style,
                           highlight_function=highlight_style).add_to(m)

    sus_alpha = 0.3

    wns_centroid_subset = wns_centroid[wns_centroid.map_year <= this_year]
    for row in wns_centroid_subset.iterrows():
        determination = row[1]['determinat']
        geom = row[1]['geometry']

        if 'WNS' in determination:
            line_color = wns_color.value
        else:
            line_color=pd_color.value

        if 'positive' in determination:
            fillOpacity = 1.0
        elif 'negative' in determination:
            fillOpacity = 0.0
        else:
            fillOpacity = sus_alpha

    #     print(determination, geom)
        folium.Circle(
        location=[geom.y, geom.x],
        radius=10000,
        color=line_color,
        fill=True,
        fill_color=line_color,
        fill_opacity = fillOpacity,
        weight=0.5).add_to(m)


    schoharie = wns[wns.countyname == 'Schoharie'].to_crs({'init': 'epsg:4326'})
    schoharie.geometry = schoharie.geometry.centroid
    folium.Circle(
        location=[schoharie.geometry.y, schoharie.geometry.x],
        radius=50000,
        color=line_color,
        tooltip='First detected February 2006\nSchoharie Co. NY',
        fill=False,
        weight=2.0).add_to(m)

    title_html = f"""<div class='my-Title' style='position: absolute; 
    z-index:9999; 
    font-size:38px;  
    padding: 20px;
    width: 100%;
    color: white;
    background-color: #343a40;
    text-align:center;'>

    <b>White-Nose Syndrome Spread Map {this_year}-{this_year+1}</b> 
    </div>
    """
    m.get_root().html.add_child(folium.Element(title_html))

    legend_html = f"""<div class='my-legend' style='position: fixed; z-index:9999; font-size:14px;  
        bottom: 20px; right: 10px; padding: 10px;
         border:2px solid grey; background-color:rgba(255, 255, 255, 0.7);
         border-radius:6px'>
    <div class='legend-title'>Legend</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>"""

    for y in range(skip_last_years.value):
        legend_html += f"\n<li><span style='background:{year_colors[y]};opacity:0.7;'></span>{this_year-y-1}-{this_year-y}</li>"

    legend_html += f"""
        <li><span style='background:{year_colors[-1]};opacity:0.7;'></span>Counties <= {this_year-4}</li>
        <li><span style='background:{endemic_color.value};opacity:0.7;'></span>Endemic Area</li>
      </ul>
    </div>

    <p class='legend-source'>WNS Positive
    <span class="dot" style='background-color:{wns_color.value};border-color:{wns_color.value};float: right;'></span>
    </p>
    <p class='legend-source'>WNS Suspect 
     <span class="dot" style='background-color:{wns_color.value};border-color:{wns_color.value};opacity:{sus_alpha};position: relative; top: 0px; left: 0px;float: right;'></span>
     <span class="dot" style='background-color:transparent;border-color:{wns_color.value};opacity:1.0;position: relative; top: 0px; left: 13px;float: right;'></span>
    </p>
    <p class='legend-source'>WNS Negative
     <span class="dot" style='background-color:transparent;border-color:{wns_color.value};opacity:1.0;float: right;'></span>
    </p>

    <p class='legend-source'>Pd Positive
    <span class="dot" style='background-color:{pd_color.value};border-color:{pd_color.value};float: right;'></span>
    </p>
    <p class='legend-source'>Pd Presumed
     <span class="dot" style='background-color:{pd_color.value};border-color:{pd_color.value};opacity:{sus_alpha};position: relative; top: 0px; left: 0px;float: right;'></span>
     <span class="dot" style='background-color:transparent;border-color:{pd_color.value};opacity:1.0;position: relative; top: 0px; left: 13px;float: right;'></span>
    </p>
    <p class='legend-source'>Pd Negative
     <span class="dot" style='background-color:transparent;border-color:{pd_color.value};opacity:1.0;float: right;'></span>
    </p>
    </div>
    """

    legend_html += """
    <style type='text/css'>
      .my-legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .my-legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .my-legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .my-legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .my-legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .my-legend a {
        color: #777;
        }
      .dot {
            height: 12px;
            width: 12px;
            border-radius: 50%;
            border-width: 2px;
            border-style: solid;
            display: inline-block;
            } 

        }
    </style>"""
    m.get_root().html.add_child(folium.Element(legend_html))


    img_html = """
    <img src="https://sciencebase.usgs.gov/nabat/assets/NABat_long_2line_color_white.png" alt="NABat Logo" style="position: fixed; 
    z-index:9999; 
    top: 3px;
    left: 3px;
    padding: 10px;

    height: 55px;
    ">"""
    m.get_root().html.add_child(folium.Element(img_html))

    plugins.Fullscreen(
        position='bottomleft',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(m)

    minimap = plugins.MiniMap(toggle_display=True, position='bottomleft')
    minimap.add_to(m)

#     m.save('wns.html')
    return m

In [21]:
colormaps = [c for c in dir(mpl.cm) if '_' not in c]
colormaps.sort()
for cruft in ['LUTSIZE', 'ScalarMappable', 'colors', 'functools']:
    try:
        colormaps.pop(colormaps.index(cruft))
    except:
        pass

cm_dropdown = widgets.Dropdown(
    options=colormaps,
    value=colormaps[1],
    description='ColorMap:',
    disabled=False,
)

reverse = widgets.Checkbox(
    value=True,
    description='Reverse Ramp',
    disabled=False
)

def change_cm(cm, r=False):
    global color_map
    color_map = mpl.cm.get_cmap(cm)
    global year_colors
    year_colors = [mpl.colors.to_hex(color_map(c)) for c in np.linspace(0, 1, skip_last_years.value+1)]
    
    if reverse.value:
        year_colors.reverse()
    
    sns.palplot(year_colors)
    
_ = interact(change_cm, cm=cm_dropdown, r=reverse)

interactive(children=(Dropdown(description='ColorMap:', index=1, options=('Accent', 'Blues', 'BrBG', 'BuGn', '…

In [22]:
wns_color = widgets.ColorPicker(
    concise=False,
    description='Pick a color for WNS indicators',
    value='red',
    disabled=False,
    style={'description_width':'initial'}
)

pd_color = widgets.ColorPicker(
    concise=False,
    description='Pick a color for Pd indicators',
    value='orange',
    disabled=False,
    style={'description_width':'initial'}
)

endemic_color = widgets.ColorPicker(
    concise=False,
    description='Pick a color for endemic area',
    value="#b3cd9f",
    disabled=False,
    style={'description_width':'initial'}
)
widgets.HBox([wns_color, pd_color, endemic_color])


In [23]:
year_dropdown = widgets.Dropdown(
    options=[y for y in wns.map_year.unique()],
    value=wns.map_year.unique()[-1],
    description='Year of map:',
    disabled=False,
)
year_dropdown

Dropdown(description='Year of map:', index=12, options=(2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, …

In [24]:
update_button = widgets.Button(
    description='Update Map',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Update Map',
)
output = widgets.Output()
display(update_button, output)

def on_button_clicked(b):
    with output:
        global kdes
        kdes = save_kdes()
        
        global m
        m = make_map()
        clear_output()
        display(m)

update_button.on_click(on_button_clicked)

Button(button_style='success', description='Update Map', style=ButtonStyle(), tooltip='Update Map')

Output()